In [1]:
from math import gcd
import random
import sympy

In [2]:
p = 10007
q = 191

In [3]:
assert sympy.isprime(p)
assert sympy.isprime(q)

In [4]:
n = p*q
phi = (p-1)*(q-1)

In [5]:
n, phi

(1911337, 1901140)

In [6]:
def generate_block_size():
    while True:
        r = random.randint(2, n)

        if (
            # r should divide p-1 without remainder
            (p-1) % r == 0 
            # r and (p - 1) / r must be coprimes
            and gcd(r, int((p - 1) / r)) == 1
            # r and q-1 must be coprimes
            and gcd(r, q - 1)
        ) == 1:
            break
    return r

In [7]:
def generate_keys():    
    while True:
        r = generate_block_size()
        
        y = random.randint(2, n)
        if gcd(y, n) != 1:
            continue
        
        # to guarantee correct decryption
        prime_factors = sympy.factorint(r).keys()
        
        # If r is composite, it was pointed out by Fousse et al. in 2011
        # that the above conditions are insufficient to guarantee correct decryption
        decryption_guaranteed = True
        for prime_factor in prime_factors:
            # none of r's prime factor should satisfy the condition
            if pow(y, int(phi/prime_factor), n) == 1:
                decryption_guaranteed = False
        if decryption_guaranteed is False:
            # regenerate keys
            continue
        
        x = pow(y, int(phi/r), n)        
        if x != 1:
            break
            
    return r, x, y

In [8]:
r, x, y = generate_keys()

In [9]:
# public key
y, r, n

(1361714, 5003, 1911337)

In [10]:
# private key
p, q, phi, x

(10007, 191, 1901140, 1028154)

# encryption

In [11]:
# message must be member of Zr
m = 17

if m >= r:
    print(f"Warning! plaintext {m} should be member of Z_{r} but it is greater.")
    m = m % r
    print(f"So, it is restored to {m}")

In [12]:
print(f"plaintext: {m}")

plaintext: 17


In [13]:
# random key for encryption
while True:
    u = random.randint(1, n)
    if gcd(u, n) == 1:
        break

In [14]:
def encrypt(m, u):
    return (pow(y, m, n) * pow(u, r, n)) % n

In [15]:
c = encrypt(m, u)

In [16]:
assert gcd(c, n) == 1

In [17]:
print(f"ciphertext is {c}")

ciphertext is 1721828


# decryption

In [18]:
def decrypt(c):
    a = pow(c, int(phi/r), n)

    md = 0
    while True:
        if pow(x, md, n) == a:
            break
        md = md + 1
    
    return md

In [19]:
m_prime = decrypt(c)

In [20]:
print(f"restored plaintext is {m_prime}")

restored plaintext is 17


In [21]:
assert m_prime == m

# Homomorphic Features of Benaloh Cryptosystem

In [22]:
m1 = 10 % r
u1 = random.randint(0, n)

c1 = encrypt(m1, u1)

In [23]:
m2 = 17 % r
u2 = random.randint(0, n)

c2 = encrypt(m2, u2)

In [24]:
# multiplication of ciphertexts must be equal to encryption 
# of addition of plaintexts
assert (c1 * c2) % n == encrypt(m1+m2, u1*u2)

In [25]:
# or deryption of multiplication of ciphertexts must be equal 
# to the addition of plaintext as well
decrypt((c1 * c2) % n) == (m1+m2) % r 

True